# Домашнее задание 2. Основные операторы PostgreSQL

### 1. Вывести все уникальные бренды, у которых есть хотя бы один продукт со стандартной стоимостью выше 1500 долларов, и суммарными продажами не менее 1000 единиц.

```sql
SELECT DISTINCT p.brand
FROM public.product p 
JOIN public.order_items oi ON p.product_id = oi.product_id
WHERE p.standard_cost > 1500
GROUP BY p.brand
HAVING SUM(oi.quantity) >= 1000;
```

### 2. Для каждого дня в диапазоне с 2017-04-01 по 2017-04-09 включительно вывести количество подтвержденных онлайн-заказов и количество уникальных клиентов, совершивших эти заказы.

```sql
SELECT 
    order_date,
    COUNT(*) AS order_count,
    COUNT(DISTINCT customer_id) AS unique_customers
FROM public.orders
WHERE order_date BETWEEN '2017-04-01' AND '2017-04-09'
    AND online_order = TRUE
    AND order_status = 'Approved'
GROUP BY order_date
ORDER BY order_date;
```

### 3. Вывести профессии клиентов:
### из сферы IT, чья профессия начинается с Senior; 
### из сферы Financial Services, чья профессия начинается с Lead.  
### Для обеих групп учитывать только клиентов старше 35 лет. Объединить выборки с помощью UNION ALL.

```sql 
SELECT job_title
FROM public.customer
WHERE job_industry_category = 'IT' 
      AND job_title LIKE 'Senior%'
      AND EXTRACT(YEAR FROM AGE(CURRENT_DATE, dob)) > 35
UNION ALL
SELECT job_title
FROM public.customer
WHERE job_industry_category = 'Financial Services' 
      AND job_title LIKE 'Lead%'
      AND EXTRACT(YEAR FROM AGE(CURRENT_DATE, dob)) > 35
```

### 4. Вывести бренды, которые были куплены клиентами из сферы Financial Services, но не были куплены клиентами из сферы IT.

``` sql
SELECT DISTINCT p.brand
FROM public.product p
JOIN public.order_items oi ON p.product_id = oi.product_id
JOIN public.orders o ON oi.order_id = o.order_id
JOIN public.customer c ON o.customer_id = c.customer_id
WHERE c.job_industry_category = 'Financial Services'
      AND p.brand NOT IN (
    SELECT DISTINCT p2.brand
    FROM public.product p2
    JOIN public.order_items oi2 ON p2.product_id = oi2.product_id
    JOIN public.orders o2 ON oi2.order_id = o2.order_id
    JOIN public.customer c2 ON o2.customer_id = c2.customer_id
    WHERE c2.job_industry_category = 'IT'
  );
  ```

### 5. Вывести 10 клиентов (ID, имя, фамилия), которые совершили наибольшее количество онлайн-заказов (в штуках) брендов Giant Bicycles, Norco Bicycles, Trek Bicycles, при условии, что они активны и имеют оценку имущества (property_valuation) выше среднего среди клиентов из того же штата.

Комментарий: я не смогла для себя точно определить, кто является активным клиентом - Y или N, так как по логике Y - это Yes, а N = No. Но пользователей с Y оказалось слишком мало, поэтому брала в расчет другую категорию. В случае недопонимания в этом моменте прошу не снижать балл)

```sql
WITH state_avg_valuation AS (
    SELECT 
        state,
        AVG(property_valuation) AS avg_property_valuation
    FROM public.customer
    WHERE deceased_indicator = 'N'
    GROUP BY state
)
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    COUNT(DISTINCT o.order_id) AS orders_count
FROM public.customer c
JOIN public.orders o ON c.customer_id = o.customer_id
JOIN public.order_items oi ON o.order_id = oi.order_id
JOIN public.product p ON oi.product_id = p.product_id
JOIN state_avg_valuation sav ON c.state = sav.state
WHERE o.online_order = TRUE
    AND o.order_status = 'Approved'
    AND c.deceased_indicator = 'N'
    AND c.property_valuation > sav.avg_property_valuation
    AND p.brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
GROUP BY 
    c.customer_id
ORDER BY 
    orders_count DESC
LIMIT 10;
```

### 6. Вывести всех клиентов (ID, имя, фамилия), у которых нет подтвержденных онлайн-заказов за последний год, но при этом они владеют автомобилем и их сегмент благосостояния не Mass Customer.

``` sql
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name
FROM public.customer c
WHERE c.owns_car = 'Yes'
    AND c.wealth_segment != 'Mass Customer'
    AND c.customer_id NOT IN (
        SELECT DISTINCT o.customer_id
        FROM public.orders o
        WHERE o.online_order = TRUE
            AND o.order_status = 'Approved'
            AND o.order_date >= CURRENT_DATE - INTERVAL '1 year'
    );
```   

### 7. Вывести всех клиентов из сферы 'IT' (ID, имя, фамилия), которые купили 2 из 5 продуктов с самой высокой list_price в продуктовой линейке Road.

``` sql
WITH top_road_products AS (
    SELECT 
        product_id,
        list_price
    FROM public.product
    WHERE product_line = 'Road'
    ORDER BY list_price DESC
    LIMIT 5
),
customer_purchases AS (
    SELECT 
        c.customer_id,
        c.first_name,
        c.last_name,
        COUNT(DISTINCT p.product_id) AS purchased_top_products
    FROM public.customer c
    JOIN public.orders o ON c.customer_id = o.customer_id
    JOIN public.order_items oi ON o.order_id = oi.order_id
    JOIN public.product p ON oi.product_id = p.product_id
    WHERE c.job_industry_category = 'IT'
        AND p.product_id IN (SELECT product_id FROM top_road_products)
    GROUP BY c.customer_id, c.first_name, c.last_name
)
SELECT 
    customer_id,
    first_name,
    last_name
FROM customer_purchases
WHERE purchased_top_products >= 2;
```

### 8. Вывести клиентов (ID, имя, фамилия, сфера деятельности) из сфер IT или Health, которые совершили не менее 3 подтвержденных заказов в период 2017-01-01 по 2017-03-01, и при этом их общий доход от этих заказов превышает 10 000 долларов.
Разделить вывод на две группы (IT и Health) с помощью UNION.

``` sql
SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.job_industry_category,
    SUM(oi.quantity * oi.item_list_price_at_sale) AS total_revenue
FROM public.customer c
JOIN public.orders o ON c.customer_id = o.customer_id
JOIN public.order_items oi ON o.order_id = oi.order_id
WHERE c.job_industry_category = 'IT'
    AND o.order_status = 'Approved'
    AND o.order_date BETWEEN '2017-01-01' AND '2017-03-01'
GROUP BY 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.job_industry_category
HAVING 
    COUNT(DISTINCT o.order_id) >= 3
    AND SUM(oi.quantity * oi.item_list_price_at_sale) > 10000

UNION

SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.job_industry_category,
    SUM(oi.quantity * oi.item_list_price_at_sale) AS total_revenue
FROM public.customer c
JOIN public.orders o ON c.customer_id = o.customer_id
JOIN public.order_items oi ON o.order_id = oi.order_id
WHERE c.job_industry_category = 'Health'
    AND o.order_status = 'Approved'
    AND o.order_date BETWEEN '2017-01-01' AND '2017-03-01'
GROUP BY 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.job_industry_category
HAVING 
    COUNT(DISTINCT o.order_id) >= 3
    AND SUM(oi.quantity * oi.item_list_price_at_sale) > 10000
ORDER BY 
    job_industry_category, 
    total_revenue DESC;
```